# PyRK tutorial by Example


##### This follow-along example is written to follow along in VScode. This tutorial also assumes the reader has no prior knowledge of reactor physics and will include some brief explanations of some of the physics.

## Requirements and Instilation

#### i)   Cloning PyRK in VScode


To clone the official PyRK repository:

1. `Ctrl+P` to open the **Command Palette** in VS Code
2. Run the line:

   ```bash
   git clone https://github.com/pyrk/pyrk.git

If you wish to use your own fork, change the link.

### ii) Installing required libraries


PyRK requires some additional Python libraries to run simulations. A list of these libraries has been included in the source code. Once PyRK is cloned, open a new terminal and run this command in the terminal to install the required libraries:

```bash
pip install -r requirements.txt

### iii)  Installing PyRK

To install PyRK, run this line in the terminal:

```bash
python setup.py install 
```

You should be ready to start using PyRK!   

## Example Simulations

PyRK runs simulations by passing two arguments into a driver script, the input file and the output folder location. PyRK comes with several examples, so it is recommended to run some of the examples before your own to check if PyRK is installed correctly and to get a sense of how PyRK works. 

To run the 'default' example, run this line in the terminal:

```bash
python .\pyrk\driver.py --infile=examples\default\input.py --plotdir=out\default

python ./pyrk/driver.py --infile=examples/default/input.py --plotdir=out/default
```
- '.\pyrk\driver.py' locates and runs the driver.
- 'infile=examples...' locates the input (.py file) containing the reactor parameters.
- '--plotdir=out...' will create a folder to save the output data.

Change each of these components to your desired specifications when running your own simulations. The default example should only take a minute to complete.

## Tutorial By Example 

In [ ]:
from pyrk.utilities.ur import units
from pyrk import th_component as th
import math
from pyrk.materials.material import Material
from pyrk.materials.liquid_material import LiquidMaterial
from pyrk.convective_model import ConvectiveModel
from pyrk.density_model import DensityModel
from pyrk.timer import Timer

def area_sphere(r):
    assert(r >= 0 * units.meter)
    return (4.0) * math.pi * pow(r.to('meter'), 2)


def vol_sphere(r):
    assert(r >= 0 * units.meter)
    return (4. / 3.) * math.pi * pow(r.to('meter'), 3)

This tutorial will follow along with the PBFHR examples, modeled after the Pebble-Bed **F**luoride salt 
cooled **H**igh temperature **R**eactor proposed by the [University of California, Berkely](https://fhr.nuc.berkeley.edu/pb-fhr-technology/). 

Each input file has two parts, the 'user workspace' and the 'user input.' The user workspace are the simulation parameters relating to physical or geometric properties of the reactor, reactivity feedbacks, or the timer and timesteps; properties typically outside of the 'control' of the user. The 'user input' is where the user is able to introduce transients, introduce feedback mechanisms, insert reactivity, and fine-tune the ODE solvers - where the actual reactor 'kinetics' happens.

## 1)  User Workspace

### Simulation Timestep Parameters

In [ ]:
# Initial time
t0 = 0.00 * units.seconds
# Timestep
dt = 0.02 * units.seconds
# Final Time
tf = 250.0 * units.seconds
# Time to turn on feedback
t_feedback = 150.0 * units.seconds

### Thermal hydraulic params

In [ ]:
# Temperature feedbacks of reactivity
alpha_fuel = -3.19 * units.pcm / units.kelvin
alpha_mod = -0.7 * units.pcm / units.kelvin
alpha_shell = -0.7 * units.pcm / units.kelvin
alpha_cool = 0.23 * units.pcm / units.kelvin

### Initial Temperatures

In [ ]:
t_mod = (800 + 273.15) * units.kelvin
t_fuel = (800 + 273.15) * units.kelvin
t_shell = (770 + 273.15) * units.kelvin
t_cool = (650 + 273.15) * units.kelvin

# Kappa
kappa = 0.0

### Volumes and Areas

In [ ]:
n_pebbles = 470000
r_mod = 1.25 / 100.0 * units.meter
r_fuel = 1.4 / 100.0 * units.meter
r_shell = 1.5 / 100.0 * units.meter

vol_mod = vol_sphere(r_mod)
vol_fuel = vol_sphere(r_fuel) - vol_sphere(r_mod)
vol_shell = vol_sphere(r_shell) - vol_sphere(r_fuel)
vol_cool = (vol_mod + vol_fuel + vol_shell) * 0.4 / 0.6
a_pb = area_sphere(r_shell)

## 2)  User Input

In [ ]:
# Total power, Watts, thermal
power_tot = 234000000.0 * units.watt

# Timer instance, based on t0, tf, dt
ti = Timer(t0=t0, tf=tf, dt=dt, t_feedback=t_feedback)

# Number of precursor groups
n_pg = 6

# Number of decay heat groups
n_dg = 0

# Fissioning Isotope
fission_iso = "fhr"
# Spectrum
spectrum = "thermal"

# Feedbacks, False to turn reactivity feedback off. True otherwise.
feedback = True

# maximum number of internal steps that the ode solver will take
nsteps = 5000

# Flow rate and inlet temperature of the coolant
m_flow = 976.0 * units.kg / units.seconds
t_inlet = units.Quantity(600.0, units.degC)

### Reactivity Insertion

In [ ]:
from pyrk.reactivity_insertion import StepReactivityInsertion
rho_ext = StepReactivityInsertion(timer=ti,
                                  t_step=t_feedback + 10.0 * units.seconds,
                                  rho_init=0.0 * units.delta_k,
                                  rho_final=650.0 * 2 * units.pcm)

### Moderator Initialization

In [ ]:
k_mod = 17 * units.watt / (units.meter * units.kelvin)
cp_mod = 1650.0 * units.joule / (units.kg * units.kelvin)
rho_mod = DensityModel(a=1740. * units.kg / (units.meter**3), model="constant")
Moderator = Material('mod', k_mod, cp_mod, rho_mod)

### Shell Initialization

In [ ]:
k_shell = 17 * units.watt / (units.meter * units.kelvin)
cp_shell = 1650.0 * units.joule / (units.kg * units.kelvin)
rho_shell = DensityModel(a=1740. * units.kg /
                         (units.meter**3), model="constant")
Shell = Material('shell', k_shell, cp_shell, rho_shell)

### Fuel Initialization

In [ ]:
k_fuel = 15 * units.watt / (units.meter * units.kelvin)
cp_fuel = 1818.0 * units.joule / units.kg / units.kelvin
rho_fuel = DensityModel(a=2220.0 * units.kg /
                        (units.meter**3), model="constant")
Fuel = Material('fuel', k_fuel, cp_fuel, rho_fuel)

### Coolant Initialization

In [ ]:
mu0 = 0 * units.pascal * units.second
k_cool = 1 * units.watt / (units.meter * units.kelvin)
cp_cool = 2415.78 * units.joule / (units.kg * units.kelvin)
rho_cool = DensityModel(a=2415.6 *
                        units.kg /
                        (units.meter**3), b=0.49072 *
                        units.kg /
                        (units.meter**3) /
                        units.kelvin, model="linear")
cool = LiquidMaterial('cool', k_cool, cp_cool, rho_cool, mu0)

### Convection Model

In [ ]:
h_cool = ConvectiveModel(
    h0=4700.0 *
    units.watt /
    units.kelvin /
    units.meter**2,
    mat=cool,
    model='constant')

### Thermal-Hydraulic Solver Parameters

In [ ]:
mod = th.THComponent(name="mod",
                     mat=Moderator,
                     vol=vol_mod,
                     T0=t_mod,
                     alpha_temp=alpha_mod,
                     timer=ti,
                     sph=True,
                     ri=0.0 * units.meter,
                     ro=r_mod)

fuel = th.THComponent(name="fuel",
                      mat=Fuel,
                      vol=vol_fuel,
                      T0=t_fuel,
                      alpha_temp=alpha_fuel,
                      timer=ti,
                      heatgen=True,
                      power_tot=power_tot / n_pebbles,
                      sph=True,
                      ri=r_mod,
                      ro=r_fuel
                      )
shell = th.THComponent(name="shell",
                       mat=Shell,
                       vol=vol_shell,
                       T0=t_shell,
                       alpha_temp=alpha_shell,
                       timer=ti,
                       sph=True,
                       ri=r_fuel,
                       ro=r_shell)

### Mesh and Pebble Treatment

In [ ]:
# mesh size for the fuel pebble FVM calculation
l = 0.0005 * units.meter
comp_list = mod.mesh(l)
comp_list.extend(fuel.mesh(l))
comp_list.extend(shell.mesh(l))
pebble = th.THSuperComponent('pebble', t_shell, comp_list, timer=ti)
# Add convective boundary condition to the pebble
pebble.add_conv_bc('cool', h=h_cool)

cool = th.THComponent(name="cool",
                      mat=cool,
                      vol=vol_cool,
                      T0=t_cool,
                      alpha_temp=alpha_cool,
                      timer=ti)
# The coolant convects to the shell
cool.add_convection('pebble', h=h_cool, area=a_pb)
cool.add_advection('cool', m_flow / n_pebbles, t_inlet, cp=cool.cp)

components = []
for i in range(0, len(pebble.sub_comp)):
    components.append(pebble.sub_comp[i])
components.extend([pebble, cool])

## Running This Notebook

#### Run this cell to convert everything in this notebook into a ``.py`` file for PyRK to read! Just copy the absolute path of this notebook and paste it into ``notebook_path``. Make sure to keep the `r` before the path.

In [ ]:
# ignore
import nbformat
from pathlib import Path

notebook_path = r"path/to/TutorialNotebook.ipynb"
op = notebook_path.replace('.ipynb', '.py')
output_path = Path(op)

with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

python_cells = [
    cell.source for cell in nb.cells
    if cell.cell_type == 'code' and '# ignore' not in cell.source
]

with open(output_path, 'w', encoding='utf-8') as f:
    for cell in python_cells:
        f.write(cell + "\n\n")

linux = "python ./pyrk/driver.py --infile=examples/tutorial/TutorialNotebook.py --plotdir=out/tutorial"
print(linux)

python ./pyrk/driver.py --infile=examples/tutorial/TutorialNotebook.py --plotdir=out/tutorial
